In [1]:
import pandas as pd
import numpy as np

X = pd.read_csv('kmeans_data/data.csv')
y = pd.read_csv('kmeans_data/label.csv')

# Determine the number of categories in 'y'
K = len(np.unique(y))

In [2]:
K

10

In [3]:
X.iloc[0][0].dtype

/var/folders/2q/kx7vz_ps7vnf2x98hyclgh7c0000gn/T/ipykernel_56204/2842412443.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  X.iloc[0][0].dtype


dtype('int64')

In [4]:
X

,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,...,0.658,0.659,0.660,0.661,0.662,0.663,0.664,0.665,0.666,0.667
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
from sklearn.metrics.pairwise import cosine_similarity

def euclidean_distance(a, b):
#     print(a)
#     print(a[0].dtype)
#     print(b[0].dtype)
    return np.sqrt(np.sum((a - b) ** 2))

def cosine_similarity_func(a, b):
    return 1 - cosine_similarity([a], [b])[0][0]

def jaccard_similarity(a, b):
    intersection = len(set(a).intersection(set(b)))
    union = len(set(a).union(set(b)))
    return 1 - (intersection / union)

In [6]:
# print(X.columns)
# print(X.index)
# print(X.iloc[np.random.choice(X.shape[0], k, replace=False)])
# X.iloc[6083]

In [7]:
len(X)

9999

In [8]:
import sys

def k_means_clustering(data, k, distance_metric, max_iters=25):
    centroids_idx = np.random.choice(len(data), k, replace=False)
    centroids = data.iloc[centroids_idx]
    for _ in range(max_iters):
        clusters = {i: [] for i in range(k)}
        for i in range(len(data)):
            point = data.iloc[i]
            min_distance=sys.maxsize
            closest_centroid=0
            centroids.index = range(10)
            
            for i,centroid in centroids.iterrows():
                distance=distance_metric(point, centroid)
                if(distance<min_distance):
                    closest_centroid=i
                    
                    min_distance=distance
            clusters[closest_centroid].append(point)

        # Update centroids
        new_centroids_data = []
        for j in range(k):
            if clusters[j]:
                new_centroid = np.mean(clusters[j], axis=0)
                new_centroids_data.append(new_centroid)
            else:
                new_centroids_data.append(centroids.loc[j].values)
        
        centroids = pd.DataFrame(new_centroids_data, columns=data.columns)
        
    return centroids,clusters

In [10]:
def calculate_sse(centroids, clusters, distance_metric):
    sse = 0
    for i in range(len(centroids)):
        cluster_points = clusters[i]
        centroid = centroids.iloc[i]
        sse += np.sum([distance_metric(point, centroid) ** 2 for point in cluster_points])
    return sse

In [11]:
# Calculate the number of categorical values in 'y' (number of classifications)
k = len(np.unique(y))

# Run K-means clustering with Euclidean distance
euclidean_centroids, euclidean_clusters = k_means_clustering(X, k, euclidean_distance)

# Calculate SSE for Euclidean-K-means
sse_euclidean = calculate_sse(euclidean_centroids, euclidean_clusters, euclidean_distance)


In [12]:
sse_euclidean

25514035741.308636

In [13]:
# Run K-means clustering with Cosine similarity
cosine_centroids, cosine_clusters = k_means_clustering(X, k, cosine_similarity_func)

# Calculate SSE for Cosine-K-means
sse_cosine = calculate_sse(cosine_centroids, cosine_clusters, cosine_similarity_func)


In [14]:
sse_cosine

693.5533720434772

In [15]:
# Run K-means clustering with Jaccard similarity (for binary data)
# Note: Jaccard similarity is commonly used for binary data
jaccard_centroids, jaccard_clusters = k_means_clustering(X, k, jaccard_similarity)

# Calculate SSE for Jaccard-K-means
sse_jaccard = calculate_sse(jaccard_centroids, jaccard_clusters, jaccard_similarity)


In [16]:
# Compare the SSEs of Euclidean-K-means, Cosine-K-means, and Jaccard-K-means
print(f"SSE for Euclidean-K-means: {sse_euclidean}")
print(f"SSE for Cosine-K-means: {sse_cosine}")
print(f"SSE for Jaccard-K-means: {sse_jaccard}")

# Determine which method is better based on the SSE values
if sse_euclidean < sse_cosine and sse_euclidean < sse_jaccard:
    print("Euclidean-K-means performs better.")
elif sse_cosine < sse_euclidean and sse_cosine < sse_jaccard:
    print("Cosine-K-means performs better.")
else:
    print("Jaccard-K-means performs better.")

SSE for Euclidean-K-means: 25514035741.308636
SSE for Cosine-K-means: 693.5533720434772
SSE for Jaccard-K-means: 9971.627450012928
Cosine-K-means performs better.


In [18]:
from collections import Counter

# Function to assign cluster labels using majority voting
def assign_cluster_labels(clusters, true_labels):
    cluster_labels = {}
    for cluster_idx, cluster_points in clusters.items():
        # Convert NumPy arrays to lists for indexing
        cluster_points = list(cluster_points)
        true_labels_cluster = true_labels[cluster_points]
        
        # Perform majority voting manually
        label_counts = {}
        for label in np.unique(true_labels_cluster):
            label_counts[label] = np.sum(true_labels_cluster == label)
        
        # Check if label_counts is empty
        if label_counts:
            # Find the label with the highest count
            most_common_label = max(label_counts, key=label_counts.get)
        else:
            # If label_counts is empty, assign a default label or handle it as needed
            most_common_label = -1  # Assign a default label
        
        # Assign the most frequent label to the cluster
        cluster_labels[cluster_idx] = most_common_label
    
    return cluster_labels

def calculate_accuracy(cluster_labels, true_labels):
    predicted_labels = np.array([cluster_labels[cluster_idx] for cluster_idx in sorted(cluster_labels)])
    
    accuracy = np.mean(predicted_labels == true_labels)
    return accuracy

true_labels = np.array(y)  

# Assign cluster labels using majority voting for each method
euclidean_cluster_labels = assign_cluster_labels(euclidean_clusters, true_labels)
cosine_cluster_labels = assign_cluster_labels(cosine_clusters, true_labels)
jaccard_cluster_labels = assign_cluster_labels(jaccard_clusters, true_labels)

# Calculate predictive accuracies for each method
accuracy_euclidean = calculate_accuracy(euclidean_cluster_labels, true_labels)
accuracy_cosine = calculate_accuracy(cosine_cluster_labels, true_labels)
accuracy_jaccard = calculate_accuracy(jaccard_cluster_labels, true_labels)

# Compare the accuracies of each method
print(f"Accuracy for Euclidean-K-means: {accuracy_euclidean}")
print(f"Accuracy for Cosine-K-means: {accuracy_cosine}")
print(f"Accuracy for Jaccard-K-means: {accuracy_jaccard}")

# Determine which method has a better accuracy
best_method = max((accuracy_euclidean, "Euclidean-K-means"),
                  (accuracy_cosine, "Cosine-K-means"),
                  (accuracy_jaccard, "Jaccard-K-means"))
print(f"The method with the best accuracy is: {best_method[1]}")

Accuracy for Euclidean-K-means: 0.10321032103210322
Accuracy for Cosine-K-means: 0.10321032103210322
Accuracy for Jaccard-K-means: 0.010321032103210321
The method with the best accuracy is: Euclidean-K-means


In [19]:
def k_means_clustering(data, k, distance_metric,max_iters=500):
    centroids_idx = np.random.choice(len(data), k, replace=False)
    centroids = data.iloc[centroids_idx].reset_index(drop=True)
    
    prev_centroids = None
    prev_sse = None
    
    for iteration in range(max_iters):
        centroids_idx = np.random.choice(len(data), k, replace=False)
        centroids = data.iloc[centroids_idx]
        for _ in range(max_iters):
            clusters = {i: [] for i in range(k)}
            for i in range(len(data)):
                point = data.iloc[i]
                min_distance=sys.maxsize
                closest_centroid=0
                centroids.index = range(10)
            
                for i,centroid in centroids.iterrows():
                    distance=distance_metric(point, centroid)
                    if(distance<min_distance):
                        closest_centroid=i
                        min_distance=distance
                clusters[closest_centroid].append(point)

        # Update centroids
        new_centroids_data = []
        for j in range(k):
            if clusters[j]:
                new_centroid = np.mean(clusters[j], axis=0)
                new_centroids_data.append(new_centroid)
            else:
                new_centroids_data.append(centroids.loc[j].values)
        
        centroids = pd.DataFrame(new_centroids_data, columns=data.columns)

        
        # Calculate SSE for the current iteration
        sse = sum(np.linalg.norm(np.array(centroids.iloc[i]) - np.array(clusters[i]), axis=1).sum() for i in range(k))
        
        if((prev_centroids is not None and (centroids.values == prev_centroids.values).all()) or (prev_sse is not None and sse > prev_sse) or (iteration == max_iters - 1)): 
                break
       
        
        # Update for next iteration
        prev_centroids = centroids.copy()
        prev_sse = sse
        
    return iteration + 1


In [ ]:
iterations_euclidean = k_means_clustering(X, k, euclidean_distance)

In [ ]:
iterations_cosine = k_means_clustering(X, k, cosine_similarity_func)

In [ ]:
iterations_jaccard = k_means_clustering(X, k, jaccard_similarity)

In [ ]:
print(f"Euclidean K-means took {iterations_euclidean} iterations to converge.")
print(f"Cosine K-means took {iterations_cosine} iterations to converge.")
print(f"Jaccard K-means took {iterations_jaccard} iterations to converge.")

In [ ]:
def k_means_clustering(data, k, stopcriteria, distance_metric,max_iters=500):
    centroids_idx = np.random.choice(len(data), k, replace=False)
    centroids = data.iloc[centroids_idx].reset_index(drop=True)
    
    prev_centroids = None
    prev_sse = None
    
    for iteration in range(max_iters):
        centroids_idx = np.random.choice(len(data), k, replace=False)
        centroids = data.iloc[centroids_idx]
        for _ in range(max_iters):
            clusters = {i: [] for i in range(k)}
            for i in range(len(data)):
                point = data.iloc[i]
                min_distance=sys.maxsize
                closest_centroid=0
                centroids.index = range(10)
            
                for i,centroid in centroids.iterrows():
                    distance=distance_metric(point, centroid)
                    if(distance<min_distance):
                        closest_centroid=i
                        min_distance=distance
                clusters[closest_centroid].append(point)

        # Update centroids
        new_centroids_data = []
        for j in range(k):
            if clusters[j]:
                new_centroid = np.mean(clusters[j], axis=0)
                new_centroids_data.append(new_centroid)
            else:
                new_centroids_data.append(centroids.loc[j].values)
        
        centroids = pd.DataFrame(new_centroids_data, columns=data.columns)

        
        # Calculate SSE for the current iteration
        sse = sum(np.linalg.norm(np.array(centroids.iloc[i]) - np.array(clusters[i]), axis=1).sum() for i in range(k))
        
        if stopcriteria == 'centroid_change':
             if (prev_centroids is not None and (centroids.values == prev_centroids.values).all()):
                break
        elif stopcriteria == 'sse_increase':
             if (prev_sse is not None and sse > prev_sse):
                break
        elif stopcriteria == 'max_iterations':
             if iteration == max_iters - 1:
                break
       
        
        # Update for next iteration
        prev_centroids = centroids.copy()
        prev_sse = sse
        
    return centroids,clusters


In [ ]:
cc_centroids_euc,cc_clusters_euc = k_means_clustering(X, k, 'centroid_change',euclidean_distance)

In [ ]:
sse_centroids_euc,sse_clusters_euc = k_means_clustering(X, k, 'sse_increase',euclidean_distance)


In [ ]:
maxiter_centroids_euc,maxiter_clusters_euc = k_means_clustering(X, k, 'max_iterations',euclidean_distance)


In [ ]:
cc_centroids_cosine,cc_clusters_cosine = k_means_clustering(X, k, 'centroid_change',cosine_similarity_func)


In [ ]:
sse_centroids_cosine,sse_clusters_cosine = k_means_clustering(X, k, 'sse_increase',cosine_similarity_func)


In [ ]:
maxiter_centroids_cosine,maxiter_clusters_cosine = k_means_clustering(X, k, 'max_iterations',cosine_similarity_func)


In [ ]:
cc_centroids_jac,cc_clusters_jac = k_means_clustering(X, k, 'centroid_change',jacard_similarity)


In [ ]:
sse_centroids_jac,sse_clusters_jac = k_means_clustering(X, k, 'sse_increase',jacard_similarity)


In [ ]:
maxiter_centroids_jac,maxiter_clusters_jac = k_means_clustering(X, k, 'max_iterations',jacard_similarity)


In [ ]:
def calculate_sse(centroids, clusters, distance_metric):
    sse = 0
    for i in range(len(centroids)):
        cluster_points = clusters[i]
        centroid = centroids.iloc[i]
        sse += np.sum([distance_metric(point, centroid) ** 2 for point in cluster_points])
    return sse

In [ ]:
print(calculate_sse(cc_centroids_euc,cc_clusters_euc,euclidean_distance))

In [ ]:
print(calculate_sse(cc_centroids_cosine,cc_clusters_cosine,cosine_similarity_func))

In [ ]:
print(calculate_sse(cc_centroids_jac,cc_clusters_jac,jaccard_similarity))

In [ ]:
print(calculate_sse(sse_centroids_euc,sse_clusters_euc,euclidean_distance))
print(calculate_sse(sse_centroids_cosine,sse_clusters_cosine,cosine_similarity_func))
print(calculate_sse(sse_centroids_jaccard,sse_clusters_jaccard,jaccard_similarity))

In [ ]:
print(calculate_sse(maxiter_centroids_euc,maxiter_clusters_euc,euclidean_distance))
print(calculate_sse(maxiter_centroids_cosine,maxiter_clusters_cosine,cosine_similarity_func))
print(calculate_sse(maxiter_centroids_jaccard,maxiter_clusters_jaccard,jaccard_similarity))